#### **[초기값 설정]**

In [ ]:
import os
from dotenv import load_dotenv

# 환경변수 읽어오기
load_dotenv(override=True)  # .env 파일을 덮어쓰기 모드로 읽기

# 환경변수 불러오기
openai_key = os.getenv("OPENAI_API_KEY")
anthropic_key = os.getenv("ANTHROPIC_API_KEY")
huggingface_token = os.getenv("HUGGINGFACEHUB_API_TOKEN")

print(f"openai key values ::: {openai_key}")  # 테스트용 (실제 서비스에서는 print 금지)
print(f"anthropic key values ::: {anthropic_key}")  # 테스트용 (실제 서비스에서는 print 금지)
print(f"huggingface_token::: {huggingface_token}")  # 테스트용 (실제 서비스에서는 print 금지)
#...ddd

import pytesseract

pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"

#### **0.프롬프트 구성**

In [1]:
# 목차
str = """


#목차
1. Introdoction to 국가건강보험 심사청구시스템
    1.1 한국 건강보험 체계
        1.1.1 Governance of NHI in Korea
        1.1.2 HIRA’s Role in 4P Concept
        1.1.3 Key Strategy on Korea NHI
    1.2 심사평가시스템 표준모델
    1.3 심사평가정보시스템 Vision

2.  Service Offering
    2.1 DUR System
        2.1.1 개요
        2.1.2 흐름도
    2.2 NHIIS System
        2.2.1 개요
        2.2.2 NHIIS 데이터 및 사용자그룹
        2.2.3 건강보험 급여청구, 심사, 지급 처리 흐름
        2.2.4 e-claim

#지시
 1.목차를 기반으로 건강보험_브로셔.ppt을 생성한다.
 2.당신은 건강보험 업무 전문가이며 문서 작성에도 탁월한 능력을 가지고 있다.
 3.ppt파일의 첫장은 목차로 구성한다.
 4.


"""



'\n# 목차\n1. Introdoction to 국가건강보험 심사청구시스템\n    1.1 한국 건강보험 체계\n        1.1.1 Governance of NHI in Korea\n        1.1.2 HIRA’s Role in 4P Concept\n        1.1.3 Key Strategy on Korea NHI\n\n    1.2  심사평가시스템 표준모델\n    1.3 심사평가정보시스템 Vision\n\n2.  Service Offering\n    2.1 DUR System\n        2.1.1 개요\n        2.1.2. \n'

#### **1.PDF문서로딩 + Embedding + Vector DB 구성**

In [ ]:
import os
from dotenv import load_dotenv

# 환경변수 읽어오기
load_dotenv(override=True)  # .env 파일을 덮어쓰기 모드로 읽기

# 환경변수 불러오기
openai_key = os.getenv("OPENAI_API_KEY")
anthropic_key = os.getenv("ANTHROPIC_API_KEY")
huggingface_token = os.getenv("HUGGINGFACEHUB_API_TOKEN")

from pdf_full_loader import PDFFullLoader
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_openai import OpenAIEmbeddings
from langchain_community.document_loaders import PyPDFium2Loader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings

#PyPDFLoader 불러오기
from langchain_community.document_loaders import PyPDFLoader

# 1. PDF 로딩 및 분할
# loader = PDFFullLoader("sample.pdf")
# documents = loader.load()

# loader = PyPDFLoader(r"G:\내 드라이브\LLM-RAG-LangChain\제안요청서_바레인.pdf")
pages = loader.load_and_split()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
chunks = text_splitter.split_documents(pages)

# 2. 임베딩
# embedding = HuggingFaceEmbeddings(model_name="jhgan/ko-sroberta-multitask")
embedding_model = OpenAIEmbeddings(model='text-embedding-3-small',openai_api_key=openai_key)    

# 3. Chroma Vector DB |  FAISS DB 구성
vectordb = Chroma.from_documents(chunks, embedding_model, persist_directory="./chroma_db")
retriever = vectordb.as_retriever(search_kwargs={"k": 4})
# faiss_db = FAISS.from_documents(chunks, embedding_model)
# retriever = faiss_db.as_retriever(search_kwargs={"k": 4})

# 4. 질의응답
retriever.search("한국 건강보험 체계의 거버넌스는 어떻게 구성되어 있나요?")  # 질의응답 예시

#### **2.LLM 연결**

In [ ]:
from langchain_openai import ChatOpenAI
from langchain.chains import RetrievalQA

llm = ChatOpenAI(
    model_name = 'exaone-3.5-2.4b-instruct',
    openai_api_key="lm-studio",
    openai_api_base="http://127.0.0.1:1234/v1",
)

from langchain_anthropic import ChatAnthropic
from anthropic import Anthropic
import httpx  

# vfy_client = httpx.Client(verify=False)

# # 1. 직접 Anthropic
# client = Anthropic(api_key=anthropic_key, http_client=vfy_client)

# # 2. Langchain Anthropic 모델 호출
# llm = ChatAnthropic(
#     model_name ="claude-3-opus-20240229",
#     anthropic_api_key=anthropic_key,)

# llm._client = client

# chat._client._transport._pool._ssl_context.check_hostname = False
# chat._client._transport._pool._ssl_context.verify_mode = 0  # ssl.CERT_NONE
# llm.invoke("안녕~ 너를 소개해줄래?")

# qa_chain = RetrievalQA.from_chain_type(
#     llm=llm,
#     retriever=retriever,  # 이건 `vectorstore.as_retriever()`와 같은 객체여야 함
#     return_source_documents=True
# )

# result = qa_chain.invoke({"query": "제안요청서에 대해 설명해줘"})
# print(result["result"])

#### **3. LLM 기반 목차 생성**

In [ ]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.llms import LlamaCpp

def generate_toc(llm, sample_text):
    prompt = PromptTemplate(
        input_variables=["content"],
        template="""
        다음 문서를 바탕으로 발표용 프레젠테이션의 목차를 작성해 주세요.
        각 항목은 한 줄씩 정리해 주세요.

        {content}
        목차 : 
        1. 사업 개요
        2. 사업 목표
        3. 상세 요구사항     
        """
    )
    chain = LLMChain(llm=llm, prompt=prompt)
    result = chain.run(sample_text[:2000])  # 일부만 사용
    return [line.strip() for line in result.split("\n") if line.strip()]

def callme():
    print("callme function called")

#### **4. 목차별 자세한 내용 생성 (Retriever 기반 RAG)**

In [6]:
from langchain.chains import RetrievalQA

def generate_content_per_topic(llm, retriever, topics):
    qa_chain = RetrievalQA.from_chain_type(llm=llm, retriever=retriever)
    results = {}
    for topic in topics:
        query = f"프레젠테이션에서 '{topic}' 항목을 설명할 내용을 자세히 작성해 주세요."
        answer = qa_chain.run(query)
        results[topic] = answer
    return results

#### **5. PPT 생성 로직**

In [7]:
# pip install python-pptx
from pptx import Presentation
from pptx.util import Inches
from io import BytesIO

def create_ppt(topics, topic_contents):
    prs = Presentation()
    for topic in topics:
        slide = prs.slides.add_slide(prs.slide_layouts[1])
        slide.shapes.title.text = topic
        content = topic_contents.get(topic, "내용 준비 중")
        slide.placeholders[1].text = content[:1500]  # 너무 길면 PPT 에러

    output_path = "generated_ppt.pptx"
    prs.save(output_path)
    return output_path

#### ***전체 실행***

In [ ]:
# 1. 문서 로딩 및 Vector DB 생성
# loader = PDFFullLoader(r"C:\Users\MOON YOUNGHO\llm\llm_mission\제안요청서_바레인.pdf",chunk_size=1000, chunk_overlap=50)
# documents = loader.load()

# embedding_model = OpenAIEmbeddings(model='text-embedding-3-small',openai_api_key=openai_key)   

# vectordb = Chroma.from_documents(documents, embedding, persist_directory="./chroma_db")
# retriever = vectordb.as_retriever()
# faiss_db = FAISS.from_documents(documents, embedding_model)
# retriever = faiss_db.as_retriever()

# 2. 목차 생성
sample_text = "\n".join(doc.page_content for doc in chunks)  # 일부 사용
print(sample_text)
topics = generate_toc(llm, sample_text)
print(topics)

# 3. 각 목차별 내용 생성
topic_contents = generate_content_per_topic(llm, retriever, topics)
print("22222222222")

# 4. PPT 생성
ppt_path = create_ppt(topics, topic_contents)
print("33333333")

print(f"📊 PPT 생성 완료: {ppt_path}")
